Requirements

1. Input the data
    We want to stack the tables on top of one another, since they have the same fields in each sheet. We can do this one of 2 ways
2. Some of the fields aren't matching up as we'd expect, due to differences in spelling. Merge these fields together
3. Make a Joining Date field based on the Joining Day, Table Names and the year 2023
4. Now we want to reshape our data so we have a field for each demographic, for each new customer (help)
5. Make sure all the data types are correct for each field
6. Remove duplicates
7. If a customer appears multiple times take their earliest joining date
8. Output the data

In [2]:
import pandas as pd

In [3]:
# Input all the data

all_sheets =  pd.read_excel('Preppin Data Inputs/New Customers.xlsx',sheet_name=None)

In [4]:
# Stacking the fields and correcting for difference in spelling

def all_files(df, sheet_name):
    df.columns = ['ID', 'Joining Day', 'Demographic', 'Value']
    df['sheet_name'] = sheet_name
    return df

processed_sheets = {
    sheet_name: all_files(df, sheet_name)
    for sheet_name, df in all_sheets.items()
}

comb_df = pd.concat(processed_sheets.values(), ignore_index=True)

In [5]:
comb_df

,ID,Joining Day,Demographic,Value,sheet_name
0,490910,3,Ethnicity,White,January
1,490910,3,Date of Birth,5/23/1981,January
2,490910,3,Account Type,Basic,January
3,369221,18,Ethnicity,Black,January
4,369221,18,Date of Birth,3/4/2019,January
...,...,...,...,...,...
2965,174699,2,Date of Birth,3/13/1989,December
2966,174699,2,Account Type,Gold,December
2967,514598,28,Ethnicity,Other,December
2968,514598,28,Date of Birth,10/10/1971,December


In [6]:
comb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2970 entries, 0 to 2969
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           2970 non-null   int64 
 1   Joining Day  2970 non-null   int64 
 2   Demographic  2970 non-null   object
 3   Value        2970 non-null   object
 4   sheet_name   2970 non-null   object
dtypes: int64(2), object(3)
memory usage: 116.1+ KB


In [7]:
# Making a join date

comb_df['Joining Month'] = pd.to_datetime(comb_df['sheet_name'], format='%B').dt.month

In [8]:
comb_df

,ID,Joining Day,Demographic,Value,sheet_name,Joining Month
0,490910,3,Ethnicity,White,January,1
1,490910,3,Date of Birth,5/23/1981,January,1
2,490910,3,Account Type,Basic,January,1
3,369221,18,Ethnicity,Black,January,1
4,369221,18,Date of Birth,3/4/2019,January,1
...,...,...,...,...,...,...
2965,174699,2,Date of Birth,3/13/1989,December,12
2966,174699,2,Account Type,Gold,December,12
2967,514598,28,Ethnicity,Other,December,12
2968,514598,28,Date of Birth,10/10/1971,December,12


In [9]:
comb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2970 entries, 0 to 2969
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             2970 non-null   int64 
 1   Joining Day    2970 non-null   int64 
 2   Demographic    2970 non-null   object
 3   Value          2970 non-null   object
 4   sheet_name     2970 non-null   object
 5   Joining Month  2970 non-null   int32 
dtypes: int32(1), int64(2), object(3)
memory usage: 127.7+ KB


In [10]:
comb_df['Joining Date'] = comb_df['Joining Day'].astype(str) + '/' + comb_df['Joining Month'].astype(str) + '/' + '2023'

In [11]:
comb_df

,ID,Joining Day,Demographic,Value,sheet_name,Joining Month,Joining Date
0,490910,3,Ethnicity,White,January,1,3/1/2023
1,490910,3,Date of Birth,5/23/1981,January,1,3/1/2023
2,490910,3,Account Type,Basic,January,1,3/1/2023
3,369221,18,Ethnicity,Black,January,1,18/1/2023
4,369221,18,Date of Birth,3/4/2019,January,1,18/1/2023
...,...,...,...,...,...,...,...
2965,174699,2,Date of Birth,3/13/1989,December,12,2/12/2023
2966,174699,2,Account Type,Gold,December,12,2/12/2023
2967,514598,28,Ethnicity,Other,December,12,28/12/2023
2968,514598,28,Date of Birth,10/10/1971,December,12,28/12/2023


In [12]:
# Reshaping data to have a field for each demographic

pivot_df = comb_df.pivot(index=['ID', 'Joining Date'], columns='Demographic', values='Value').reset_index()

In [13]:
pivot_df

Demographic,ID,Joining Date,Account Type,Date of Birth,Ethnicity
0,100185,20/5/2023,Basic,7/29/1952,Asian
1,101515,14/4/2023,Gold,8/11/1974,Black
2,101744,29/8/2023,Basic,1/21/1945,Asian
3,102704,23/1/2023,Basic,3/9/2000,Black
4,103488,28/8/2023,Basic,9/26/1957,Other
...,...,...,...,...,...
985,994016,29/3/2023,Platinum,3/29/1955,Other
986,994289,16/6/2023,Gold,5/9/1990,White
987,994611,10/1/2023,Basic,6/19/1994,Black
988,995456,17/2/2023,Basic,3/5/1975,Other


In [14]:
# Make sure all data types are correct for each field

pivot_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990 entries, 0 to 989
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             990 non-null    int64 
 1   Joining Date   990 non-null    object
 2   Account Type   990 non-null    object
 3   Date of Birth  990 non-null    object
 4   Ethnicity      990 non-null    object
dtypes: int64(1), object(4)
memory usage: 38.8+ KB


In [15]:
pivot_df['Date of Birth'] = pd.to_datetime(pivot_df['Date of Birth'], format='%m/%d/%Y')
pivot_df['Joining Date'] = pd.to_datetime(pivot_df['Joining Date'], format='%d/%m/%Y')

In [16]:
pivot_df

Demographic,ID,Joining Date,Account Type,Date of Birth,Ethnicity
0,100185,2023-05-20,Basic,1952-07-29,Asian
1,101515,2023-04-14,Gold,1974-08-11,Black
2,101744,2023-08-29,Basic,1945-01-21,Asian
3,102704,2023-01-23,Basic,2000-03-09,Black
4,103488,2023-08-28,Basic,1957-09-26,Other
...,...,...,...,...,...
985,994016,2023-03-29,Platinum,1955-03-29,Other
986,994289,2023-06-16,Gold,1990-05-09,White
987,994611,2023-01-10,Basic,1994-06-19,Black
988,995456,2023-02-17,Basic,1975-03-05,Other


In [17]:
# Removing Duplicates
pivot_df = pivot_df.drop_duplicates(subset='ID')

In [18]:
pivot_df

Demographic,ID,Joining Date,Account Type,Date of Birth,Ethnicity
0,100185,2023-05-20,Basic,1952-07-29,Asian
1,101515,2023-04-14,Gold,1974-08-11,Black
2,101744,2023-08-29,Basic,1945-01-21,Asian
3,102704,2023-01-23,Basic,2000-03-09,Black
4,103488,2023-08-28,Basic,1957-09-26,Other
...,...,...,...,...,...
985,994016,2023-03-29,Platinum,1955-03-29,Other
986,994289,2023-06-16,Gold,1990-05-09,White
987,994611,2023-01-10,Basic,1994-06-19,Black
988,995456,2023-02-17,Basic,1975-03-05,Other


In [19]:
# Output the Data

pivot_df.to_csv('Preppin Data Outputs/pd2023wk4_output.csv', index= False)